In [0]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable

In [0]:
%run /Workspace/Consolidated_pipeline/1_setup/utilities

In [0]:
dbutils.widgets.text("catalog", "fmcg", "Catalog")
dbutils.widgets.text("data_source", "orders", "Data Source")

catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

base_path = f's3://sportsbar-dj047/{data_source}'
landing_path = f'{base_path}/landing'
processed_path = f'{base_path}/processed'
print("Base_path : ",base_path)
print("Landing_path : ",landing_path)
print("Processed_path : ",processed_path)

bronze_table = f'{catalog}.{bronze_schema}.{data_source}'
silver_table = f'{catalog}.{silver_schema}.{data_source}'
gold_table = f'{catalog}.{gold_schema}.sb_fact_{data_source}'


In [0]:
df = spark.read.format('csv').option('header',True).option('inferschema',True).load(f'{landing_path}/*.csv')\
    .withColumn('read_timestamp',current_timestamp()).select("*", '_metadata.file_name','_metadata.file_size')

df.limit(10).display(truncate=False)

In [0]:
df.write.format('delta').mode('append').option('enableChangeDataFeed',True).saveAsTable(bronze_table)

In [0]:
files = dbutils.fs.ls(landing_path) 

for files_info in files:
  dbutils.fs.mv(
      files_info.path,
      f"{processed_path}/{files_info.name}",
      True
  )

Silver Data Processing

In [0]:
df_bronze = spark.sql(f"SELECT * FROM {bronze_table}")
df_bronze.limit(10).display(truncate=False)

In [0]:
df_bronze.filter(col('order_qty').isNull()).count()

1. drop null order quantity

In [0]:
print("No of rows before null order_quantity removal : ",df_bronze.count())
df_bronze = df_bronze.filter(col('order_qty').isNotNull())
print("No of rows after null order_quantity removal :",df_bronze.count())

2. clean Customer_id (keep numeric else set to 999999)

In [0]:
df_bronze = df_bronze.withColumn('customer_id',when(col('customer_id').rlike('^[0-9]+$'),col('customer_id')).otherwise('999999').cast('string'))
df_bronze.limit(10).display(truncate=False)

3. Remove weekday name from date text 
(Tuesday, July 01, 2025 -> July 01, 2025)

In [0]:
df_bronze = df_bronze.withColumn("order_placement_date",regexp_replace(col("order_placement_date"), r"^[A-Za-z]+,\s*", ""))
df_bronze.limit(10).display(truncate=False)


4. Parse order_placement_date using multiple possible format

In [0]:
df_bronze = df_bronze.withColumn(
    "order_placement_date",
    coalesce(
        try_to_date(col("order_placement_date"), "yyyy/MM/dd"),
        try_to_date(col("order_placement_date"), "dd-MM-yyyy"),
        try_to_date(col("order_placement_date"), "dd/MM/yyyy"),
        try_to_date(col("order_placement_date"), "MMMM dd, yyyy")
    )
)

In [0]:
df_bronze.limit(10).display(truncate=False)

5. Drop Duplicates


In [0]:
df_bronze = df_bronze.dropDuplicates(['order_id','order_placement_date','customer_id','product_id','order_qty'])

6. convert product_id to string

In [0]:
df_bronze = df_bronze.withColumn('product_id',col('product_id').cast('string'))
df_bronze.limit(10).display(truncate=False)

In [0]:
# check maximum date

df_bronze.agg(max('order_placement_date').alias('max_date'),min('order_placement_date').alias('min_date')).display(truncate=False)

In [0]:
df_products = spark.table('fmcg.silver.products')

In [0]:
df_products.display(trucnate=False)

In [0]:
df_joined = df_bronze.join(df_products,df_bronze.product_id == df_products.product_id,'inner').select(df_bronze['*'],df_products['product_code'])

display(df_joined.limit(10))

In [0]:
if not (spark.catalog.tableExists(silver_table)):
    df_joined.write.format('delta').option('delta.enableChangeDataFeed', 'true').option('merge_schema','true').mode('overwrite').saveAsTable(silver_table)
else:
    silver_delta = DeltaTable.forName(spark, silver_table)
    silver_delta.alias("silver").merge(df_joined.alias("bronze"), "silver.order_placement_date = bronze.order_placement_date AND silver.order_id = bronze.order_id AND silver.product_code = bronze.product_code AND silver.customer_id = bronze.customer_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

Gold Data Processing

In [0]:
df_gold = spark.sql(f"SELECT order_id, order_placement_date as date, customer_id as customer_code, product_code, product_id, order_qty as sold_quantity FROM {silver_table};")

df_gold.show(2)

In [0]:
if not (spark.catalog.tableExists(gold_table)):
    print("creating New Table")
    df_gold.write.format("delta").option(
        "delta.enableChangeDataFeed", "true"
    ).option("mergeSchema", "true").mode("overwrite").saveAsTable(gold_table)
else:
    gold_delta = DeltaTable.forName(spark, gold_table)
    gold_delta.alias("source").merge(df_gold.alias("gold"), "source.date = gold.date AND source.order_id = gold.order_id AND source.product_code = gold.product_code AND source.customer_code = gold.customer_code").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

Merging with Parent Company

* We want data on monthly level but child data is on monthly level

In [0]:
# Full load

df_child = spark.sql(f"SELECT date, product_code, customer_code, sold_quantity FROM {gold_table}")
df_child.show(10)

In [0]:
df_child.count()

In [0]:
df_monthly = (
    df_child
    # 1. Get month start date (e.g., 2025-11-30 → 2025-11-01)
    .withColumn("month_start", trunc("date", "MM"))   # or F.date_trunc("month", "date").cast("date")

    # 2.Group at monthly grain by month_start + product_code + customer_code
    .groupBy("month_start", "product_code", "customer_code")
    .agg(
        sum("sold_quantity").alias("sold_quantity")
    )

    # 3. Rename month_start back to `date` to match your target schema
    .withColumnRenamed("month_start", "date")
)

df_monthly.show(5, truncate=False)

In [0]:
df_monthly.count()

In [0]:
gold_parent_delta = DeltaTable.forName(spark, f"{catalog}.{gold_schema}.fact_orders")
gold_parent_delta.alias("parent_gold").merge(df_monthly.alias("child_gold"), "parent_gold.date = child_gold.date AND parent_gold.product_code = child_gold.product_code AND parent_gold.customer_code = child_gold.customer_code").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()